In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("CShorten/ML-ArXiv-Papers")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'title', 'abstract'],
        num_rows: 117592
    })
})

In [4]:
split_datasets = dataset["train"].train_test_split(test_size=0.2)

In [5]:
split_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'title', 'abstract'],
        num_rows: 94073
    })
    test: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'title', 'abstract'],
        num_rows: 23519
    })
})

In [6]:
dataset_train = split_datasets["train"]

In [7]:
dataset_train

Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'title', 'abstract'],
    num_rows: 94073
})

In [8]:
dataset_train = dataset_train.remove_columns(['Unnamed: 0','Unnamed: 0.1'])

In [9]:
dataset_train

Dataset({
    features: ['title', 'abstract'],
    num_rows: 94073
})

In [10]:
dataset_train[0]

{'title': 'Catching Up Faster by Switching Sooner: A Prequential Solution to the\n  AIC-BIC Dilemma',
 'abstract': '  Bayesian model averaging, model selection and its approximations such as BIC\nare generally statistically consistent, but sometimes achieve slower rates og\nconvergence than other methods such as AIC and leave-one-out cross-validation.\nOn the other hand, these other methods can br inconsistent. We identify the\n"catch-up phenomenon" as a novel explanation for the slow convergence of\nBayesian methods. Based on this analysis we define the switch distribution, a\nmodification of the Bayesian marginal distribution. We show that, under broad\nconditions,model selection and prediction based on the switch distribution is\nboth consistent and achieves optimal convergence rates, thereby resolving the\nAIC-BIC dilemma. The method is practical; we give an efficient implementation.\nThe switch distribution has a data compression interpretation, and can thus be\nviewed as a "prequ

In [11]:
dataset_train = dataset_train.map(lambda x, idx: { 'index': idx }, with_indices=True)

Map:   0%|          | 0/94073 [00:00<?, ? examples/s]

In [12]:
dataset_train

Dataset({
    features: ['title', 'abstract', 'index'],
    num_rows: 94073
})

In [13]:
dataset_train[4]

{'title': 'An Experimental Design Perspective on Model-Based Reinforcement Learning',
 'abstract': '  In many practical applications of RL, it is expensive to observe state\ntransitions from the environment. For example, in the problem of plasma control\nfor nuclear fusion, computing the next state for a given state-action pair\nrequires querying an expensive transition function which can lead to many hours\nof computer simulation or dollars of scientific research. Such expensive data\ncollection prohibits application of standard RL algorithms which usually\nrequire a large number of observations to learn. In this work, we address the\nproblem of efficiently learning a policy while making a minimal number of\nstate-action queries to the transition function. In particular, we leverage\nideas from Bayesian optimal experimental design to guide the selection of\nstate-action queries for efficient learning. We propose an acquisition function\nthat quantifies how much information a state-act

In [14]:
dataset_train[20572]

{'title': 'Efficient Motion Planning for Automated Lane Change based on Imitation\n  Learning and Mixed-Integer Optimization',
 'abstract': '  Intelligent motion planning is one of the core components in automated\nvehicles, which has received extensive interests. Traditional motion planning\nmethods suffer from several drawbacks in terms of optimality, efficiency and\ngeneralization capability. Sampling based methods cannot guarantee the\noptimality of the generated trajectories. Whereas the optimization-based\nmethods are not able to perform motion planning in real-time, and limited by\nthe simplified formalization. In this work, we propose a learning-based\napproach to handle those shortcomings. Mixed Integer Quadratic Problem based\noptimization (MIQP) is used to generate the optimal lane-change trajectories\nwhich served as the training dataset for learning-based action generation\nalgorithms. A hierarchical supervised learning model is devised to make the\nfast lane-change decisi

In [15]:
import numpy as np
all_indexes = np.array(dataset_train['index'])

In [16]:
import random

def generate_hard_negatives(example, num_negatives=10,dataset=dataset_train):
    query_index = example['index'] 
    anchor = example['title']
    positive = example['abstract']

    negatives = []
    
    negative_indexes = np.delete(all_indexes, np.where(all_indexes == query_index))
    
    sampled_negatives = random.sample(list(negative_indexes), num_negatives)
    
    for idx in sampled_negatives:
        negatives.append(dataset[int(idx)]['abstract'])
    
    return {
        "query": anchor,
        "positive": positive,
        "negatives": negatives
    }

In [17]:
processed_data_train = dataset_train.map(generate_hard_negatives, remove_columns=dataset_train.column_names)

Map:   0%|          | 0/94073 [00:00<?, ? examples/s]

In [18]:
processed_data_train

Dataset({
    features: ['query', 'positive', 'negatives'],
    num_rows: 94073
})

In [19]:
processed_data_train[0]

{'query': 'Catching Up Faster by Switching Sooner: A Prequential Solution to the\n  AIC-BIC Dilemma',
 'positive': '  Bayesian model averaging, model selection and its approximations such as BIC\nare generally statistically consistent, but sometimes achieve slower rates og\nconvergence than other methods such as AIC and leave-one-out cross-validation.\nOn the other hand, these other methods can br inconsistent. We identify the\n"catch-up phenomenon" as a novel explanation for the slow convergence of\nBayesian methods. Based on this analysis we define the switch distribution, a\nmodification of the Bayesian marginal distribution. We show that, under broad\nconditions,model selection and prediction based on the switch distribution is\nboth consistent and achieves optimal convergence rates, thereby resolving the\nAIC-BIC dilemma. The method is practical; we give an efficient implementation.\nThe switch distribution has a data compression interpretation, and can thus be\nviewed as a "prequ

In [20]:
contrastive_pairs_train = []
for item in processed_data_train:
    query = item["query"]
    positive = item["positive"]
    negatives = item["negatives"]
    contrastive_pairs_train.append({
        "anchor": query,
        "positive": positive,
        "negatives": negatives
    })

In [21]:
len(contrastive_pairs_train)

94073

In [22]:
from torch.utils.data import DataLoader

In [23]:
class ContrastiveDataset:
    def __init__(self, pairs):
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        item = self.pairs[idx]
        return item["anchor"], item["positive"], item["negatives"]

In [24]:
contrastive_dataset_train = ContrastiveDataset(contrastive_pairs_train)

In [25]:
data_loader_train = DataLoader(contrastive_dataset_train, batch_size=8, shuffle=True)

In [26]:
len(data_loader_train)

11760

In [27]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [28]:
model = AutoModel.from_pretrained("bert-base-uncased")

In [29]:
from peft import LoraConfig, get_peft_model

In [30]:
lora_config = LoraConfig(
    task_type= "FEATURE_EXTRACTION"
)

In [31]:
lora_model = get_peft_model(model, lora_config)

In [32]:
def lorentzian_distance(x, y):
    
    dot_product = torch.sum(x * y, dim=-1)
    norm_x = torch.norm(x, dim=-1)
    norm_y = torch.norm(y, dim=-1)
    
    distance = torch.acosh(-dot_product + torch.sqrt((1 + norm_x**2) * (1 + norm_y**2)))
    return distance

In [33]:
def info_nce_loss(anchor_embedding, positive_embedding, negative_embedding, distance_fn):

    pos_dist = distance_fn(anchor_embedding, positive_embedding)
    neg_dist = torch.stack([distance_fn(anchor_embedding, neg) for neg in negative_embedding], dim=-1)
    
    logits = torch.cat([-pos_dist.unsqueeze(1), -neg_dist], dim=1)
    labels = torch.zeros(logits.size(0), dtype=torch.long, device=logits.device)

    loss = torch.nn.CrossEntropyLoss()(logits, labels)
    return loss

In [34]:
def exterior_angle(x_space, y_space, c):
    norm_x_space = torch.norm(x_space, p=2, dim=-1)
    norm_y_space = torch.norm(y_space, p=2, dim=-1)
    x_time = torch.sqrt(1/c + norm_x_space**2)
    y_time = torch.sqrt(1/c + norm_y_space**2)
    dot_product = torch.sum(x_space * y_space, dim=-1)
    lorentz_inner_product =  dot_product - x_time * y_time
    numerator = y_time + x_time * c * lorentz_inner_product
    denominator = norm_x_space * torch.sqrt((c * lorentz_inner_product)**2 - 1)
    ext_angle = torch.acos(numerator / denominator)
    return ext_angle

In [35]:
def entailment_loss(x, y, c=1, K=0.1):
    c = torch.tensor(c)
    K = torch.tensor(K)
    xspace = x
    yspace = y
    aperture = torch.asin(2 * K / (torch.sqrt(c) * torch.norm(xspace, p=2, dim=-1)))
    
    ext_angle = exterior_angle(xspace,yspace,c=c)
    
    loss = torch.max(torch.zeros_like(ext_angle), ext_angle - aperture)
    return loss.mean()

In [36]:
def expm_o(v, c=1.0):
    c = torch.tensor(c)
    vspace = v
    vnorm = torch.norm(v, p=2, dim=-1, keepdim=True)
    xspace = torch.sinh(torch.sqrt(c) * vnorm) * vspace / (torch.sqrt(c) * vnorm)
    batch_min = xspace.min(dim=1, keepdim=True).values
    batch_max = xspace.max(dim=1, keepdim=True).values
    xspace_scaled=(xspace - batch_min) / (batch_max - batch_min)
    return xspace_scaled

In [37]:
import torch
import torch.optim as optim
optimizer = torch.optim.AdamW(lora_model.parameters(), lr=5e-5)

In [38]:
num_epochs=3

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [40]:
device

device(type='cuda')

In [41]:
torch.cuda.is_available()

True

In [42]:
lora_model = lora_model.to(device)

In [43]:
dataset_val = split_datasets["test"]

In [44]:
dataset_val

Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'title', 'abstract'],
    num_rows: 23519
})

In [45]:
dataset_val = dataset_val.remove_columns(['Unnamed: 0','Unnamed: 0.1'])

In [46]:
dataset_val = dataset_val.map(lambda x, idx: { 'index': idx }, with_indices=True)

Map:   0%|          | 0/23519 [00:00<?, ? examples/s]

In [47]:
dataset_val

Dataset({
    features: ['title', 'abstract', 'index'],
    num_rows: 23519
})

In [48]:
all_indexes_val = np.array(dataset_val['index'])

In [49]:
def generate_hard_negatives_val(example, num_negatives=10,dataset=dataset_val):
    query_index = example['index'] 
    anchor = example['title']
    positive = example['abstract']
    
    negatives = []
    
    negative_indexes = np.delete(all_indexes_val, np.where(all_indexes_val == query_index))
    
    sampled_negatives = random.sample(list(negative_indexes), num_negatives)
    
    for idx in sampled_negatives:
        negatives.append(dataset[int(idx)]['abstract'])
    
    return {
        "query": anchor,
        "positive": positive,
        "negatives": negatives
    }

In [50]:
processed_data_val = dataset_val.map(generate_hard_negatives_val, remove_columns=dataset_val.column_names)

Map:   0%|          | 0/23519 [00:00<?, ? examples/s]

In [51]:
contrastive_pairs_val = []
for item in processed_data_val:
    query = item["query"]
    positive = item["positive"]
    negatives = item["negatives"]
    contrastive_pairs_val.append({
        "anchor": query,
        "positive": positive,
        "negatives": negatives
    })

In [52]:
contrastive_dataset_val = ContrastiveDataset(contrastive_pairs_val)

In [53]:
data_loader_val = DataLoader(contrastive_dataset_val, batch_size=8, shuffle=True)

In [54]:
len(data_loader_val)

2940

In [55]:
def evaluate_mrr(model1, data_loader_val, distance_fn):
    model1.eval()
    
    total_rr = 0.0
    num_queries = 0

    with torch.no_grad():
        for batch in data_loader_val:
            anchor_text = batch[0]
            positive_text = batch[1]
            negative_texts = batch[2]

            anchor_input = tokenizer(anchor_text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
            positive_input = tokenizer(positive_text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)

            anchor_embedding = expm_o(model1(**anchor_input).last_hidden_state[:, 0, :])
            positive_embedding = expm_o(model1(**positive_input).last_hidden_state[:, 0, :])
            negative_embedding = [expm_o(model1(**tokenizer(neg, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)).last_hidden_state[:, 0, :]) for neg in negative_texts]

            pos_dist = distance_fn(anchor_embedding, positive_embedding)
            neg_dist = torch.stack([distance_fn(anchor_embedding, neg) for neg in negative_embedding], dim=-1)
            all_similarities=torch.cat([-pos_dist.unsqueeze(1), -neg_dist], dim=1)

            sorted_similarities, sorted_indices = torch.sort(all_similarities, dim=1, descending=True)

            # Find the rank of the first relevant (positive) document
            positive_rank = (sorted_indices == 0).nonzero(as_tuple=True)[1] + 1  # +1 to make rank 1-based
            total_rr += torch.sum(1.0 / positive_rank.float()).item()  # Reciprocal rank
            num_queries += len(positive_rank)
            
    mrr = total_rr / num_queries
    return mrr

In [56]:
import os
save_dir ="/dss/dsshome1/07/ra65bex2/srawat/0.9hyperbolic"

In [57]:
import time
epoch_metrics = []

In [58]:
torch.cuda.empty_cache()

In [59]:
for epoch in range(num_epochs):
    start_time = time.time()
    lora_model.train()

    total_loss = 0.0
    entailment_loss_total=0.0
    contrastive_loss_total=0.0
    for batch in data_loader_train:

        anchor_texts = batch[0]
        positive_texts = batch[1]
        negative_texts = batch[2]

        anchor_inputs = tokenizer(anchor_texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
        positive_inputs = tokenizer(positive_texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    
        anchor_embedding = expm_o(lora_model(**anchor_inputs).last_hidden_state[:, 0, :])
        positive_embedding = expm_o(lora_model(**positive_inputs).last_hidden_state[:, 0, :])
        negative_embedding = [expm_o(lora_model(**tokenizer(neg, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)).last_hidden_state[:, 0, :]) for neg in negative_texts]

        contrastive_loss_value = info_nce_loss(anchor_embedding, positive_embedding, negative_embedding, distance_fn=lorentzian_distance)
        
        entailment_loss_value = entailment_loss(anchor_embedding, positive_embedding)
        
        loss = contrastive_loss_value + 0.9*entailment_loss_value
  
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        entailment_loss_total+=entailment_loss_value.item()
        contrastive_loss_total+=contrastive_loss_value.item()
    save_path1 = os.path.join(save_dir, f"hyperbolic_lora_checkpoint_epoch_{epoch+1}.pth")
    torch.save(lora_model, save_path1)
    print(f"EPOCH {epoch+1}:")
    print(f"Checkpoint saved: {save_dir}")
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(data_loader_train)}")
    print(f"Epoch {epoch+1}/{num_epochs}, Contrastive Loss: {contrastive_loss_total / len(data_loader_train)}")
    print(f"Epoch {epoch+1}/{num_epochs}, Entailment Loss: {entailment_loss_total / len(data_loader_train)}")
    mrr_validation = evaluate_mrr(model1=lora_model, data_loader_val=data_loader_val,distance_fn=lorentzian_distance)
    #mrr_train = evaluate_mrr(lora_model, data_loader_train, lorentzian_distance)
    #print(f"Mean Reciprocal Rank (MRR) for training set: {mrr_train:.4f}")
    print(f"Mean Reciprocal Rank (MRR) for validation set: {mrr_validation:.4f}")
    end_time = time.time()
    print(f"Epoch {epoch+1} took {(end_time - start_time) / 60:.4f} minutes.")
    print(f"\n")
    epoch_metrics.append({
        'epoch': epoch + 1,
        'training_loss': total_loss / len(data_loader_train),
        'Contrastive_loss': contrastive_loss_total / len(data_loader_train),
        'Entailment_loss': entailment_loss_total / len(data_loader_train),
        'mrr_validation': mrr_validation,
        'time_taken_minutes': (end_time - start_time) / 60
    })

EPOCH 1:
Checkpoint saved: /dss/dsshome1/07/ra65bex2/srawat/0.9hyperbolic
Epoch 1/3, Loss: 3.4693303145721655
Epoch 1/3, Contrastive Loss: 2.4000808057330905
Epoch 1/3, Entailment Loss: 1.1880550388373485


Mean Reciprocal Rank (MRR) for validation set: 0.2765
Epoch 1 took 181.1570 minutes.




EPOCH 2:
Checkpoint saved: /dss/dsshome1/07/ra65bex2/srawat/0.9hyperbolic
Epoch 2/3, Loss: 3.348172242098114
Epoch 2/3, Contrastive Loss: 2.398834418297625
Epoch 2/3, Entailment Loss: 1.0548198314852455


Mean Reciprocal Rank (MRR) for validation set: 0.2772
Epoch 2 took 181.3225 minutes.




EPOCH 3:
Checkpoint saved: /dss/dsshome1/07/ra65bex2/srawat/0.9hyperbolic
Epoch 3/3, Loss: 3.318083469883925
Epoch 3/3, Contrastive Loss: 2.39853411164819
Epoch 3/3, Entailment Loss: 1.0217215355764442


Mean Reciprocal Rank (MRR) for validation set: 0.2755
Epoch 3 took 181.0848 minutes.




In [60]:
import json

In [61]:
with open(save_dir + '/epoch_metrics.json', 'w') as f:
    json.dump(epoch_metrics, f)